In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('default')     #switch betweeb 'default' and 'ignore'

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True


In [2]:
import os
import sys
import mindsdb_sdk

# os.environ['KMP_DUPLICATE_LIB_OK']='True'
mod_dir = os.path.abspath(os.pardir).split('notebooks/')[0]
sys.path.insert(1,mod_dir)
# from utils.modules.ml.timeseries import rollingstats as rs

''' restart initiate classes '''
if debug:
    import importlib
#     rs = importlib.reload(rs)
    
__desc__ = "example testing mindsdb"

# clsRS = rs.RollingStats(desc=__desc__)
print("\nClass initialization and load complete!")


Class initialization and load complete!


## Read sample data

In [3]:
import pandas as pd

_data_fpath=os.path.join(mod_dir,
                         "data/ml/timeseries/sampledata",
                         "rollingstats.csv")
_test_data_df = pd.read_csv(_data_fpath)
col_list = ['asset_name','mcap_date','mcap_value',]
_test_data_df=_test_data_df[col_list]
_test_data_df

,asset_name,mcap_date,mcap_value
0,gamezone,2023-03-02 00:00:00,1.586919e+07
1,the_virtua_kolect,2023-03-02 00:00:00,2.200934e+07
2,alchemy_pay,2023-03-02 00:00:00,1.807773e+08
3,ergo,2023-03-02 00:00:00,1.055061e+08
4,handshake,2023-03-02 00:00:00,2.328368e+07
...,...,...,...
15696,ecox,2023-03-17 00:00:00,2.011757e+07
15697,phantasma,2023-03-17 00:00:00,2.150946e+07
15698,hermez_network,2023-03-17 00:00:00,8.430363e+07
15699,sero,2023-03-17 00:00:00,1.177103e+07


In [5]:
# check_container.py
from typing import Optional

import docker

def is_container_running(container_name: str) -> Optional[bool]:
    """Verify the status of a container by it's name

    :param container_name: the name of the container
    :return: boolean or None
    """
    RUNNING = "running"
    # Connect to Docker using the default socket or the configuration
    # in your environment
    docker_client = docker.from_env()
    # Or give configuration
    # docker_socket = "unix://var/run/docker.sock"
    # docker_client = docker.DockerClient(docker_socket)

    try:
        container = docker_client.containers.get(container_name)
    except docker.errors.NotFound as exc:
        print(f"Check container name!\n{exc.explanation}")
    else:
        container_state = container.attrs["State"]
        return container_state["Status"] == RUNNING


# if __name__ == "__main__":
container_name = "sleepy_banach"
result = is_container_running(container_name)
print(result)

True


/tmp/ipykernel_8566/1647426924.py:31: ResourceWarning: unclosed <socket.socket fd=57, family=AddressFamily.AF_UNIX, type=SocketKind.SOCK_STREAM, proto=0, raddr=/run/docker.sock>
  result = is_container_running(container_name)
/tmp/ipykernel_8566/1647426924.py:31: ResourceWarning: unclosed <socket.socket fd=59, family=AddressFamily.AF_UNIX, type=SocketKind.SOCK_STREAM, proto=0, raddr=/run/docker.sock>
  result = is_container_running(container_name)


In [6]:
# connects to the specified host and port
server = mindsdb_sdk.connect('http://127.0.0.1:47334')
server

Server(http://127.0.0.1:47334)

## get project

In [7]:
__def_proj_name__ = 'rez_test_proj'

if __def_proj_name__ not in [p.name for p in server.list_projects()]:
    project = server.create_project(__def_proj_name__)
project = server.get_project(__def_proj_name__)
project

Project(rez_test_proj)

## Get files

In [26]:
fname = "coin_mcap"
f_list=server.get_database("files")
if fname in [f.name for f in f_list.list_tables()]:
    tbl=f_list.get_table(fname)
    print("%s %s retrieved\n" % (fname,type(tbl)))
    print(tbl.limit(3).fetch())
else:
    print("No table %s" % fname)

coin_mcap <class 'mindsdb_sdk.query.Table'> retrieved

   mcap_past_pk                      uuid data_source         asset_name  \
0         46400  64538e084a6e0a5e3a6f7acb   coingecko           gamezone   
1         41977  6453d10d4a6e0a5e3a805a02   coingecko  the_virtua_kolect   
2         40238  6453638f4a6e0a5e3a646bb1   coingecko        alchemy_pay   

  asset_symbol       alt_asset_id currency           price_date  price_value  \
0        gzone           gamezone      usd  2023-03-02 00:00:00     0.038417   
1          tvk  the-virtua-kolect      usd  2023-03-02 00:00:00     0.050673   
2          ach        alchemy-pay      usd  2023-03-02 00:00:00     0.036582   

   price_log_ror  ...          volume_date   volume_size  volume_change  \
0      -0.068320  ...  2023-03-02 00:00:00  1.137197e+05           None   
1      -0.064649  ...  2023-03-02 00:00:00  2.609277e+07           None   
2      -0.063371  ...  2023-03-02 00:00:00  8.878882e+07           None   

                  

/tmp/ipykernel_434127/2140251015.py:6: ResourceWarning: unclosed <socket.socket fd=59, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 58162), raddr=('127.0.0.1', 47334)>
  print(tbl.limit(3).fetch())


## Create model

In [42]:
''' uncomment to drop model '''
__def_model_name__="mcap_price_predict"
if __def_model_name__ in [m.name for m in  project.list_models()]:
    project.drop_model(__def_model_name__)

In [45]:
__def_model_name__="mcap_price_predict"
__def_engine__= ""

if __def_model_name__ not in [m.name for m in  project.list_models()]:
    print("model : %s does not exist; creating new model" % __def_model_name__)
    mcap_model = project.create_model(
        name = __def_model_name__,
        predict='mcap_value',
        query = f_list.query(f"SELECT mcap_date, asset_name, mcap_value "+\
                f"FROM {fname};"),
        timeseries_options={
            "ORDER BY":'mcap_date',
            "GROUP BY":"asset_name",
            'WINDOW' : 5,
            'HORIZON': 1,
        }
    )
# try:
mcap_model=project.get_model(__def_model_name__)
mcap_model
# except Exception as e:
#     print(e)


model : mcap_price_predict does not exist; creating new model


Model(mcap_price_predict, status=generating)

In [18]:
print(mcap_model.get_status())
print(mcap_model.data['error'])

complete
None


In [46]:
import time

print('Waiting for model training to complete...please be patient')
for i in range(400):
    print('.', end='')
    time.sleep(0.5)

    if mcap_model.get_status() not in ('generating', 'training'):
        print(f'\nFinished training {mcap_model.name}:')
        break
# get model's details      
print(mcap_model.data)

Waiting for model training to complete...please be patient
................................................................................................................................................................................................................................................................................................................................................................................................................{'name': 'mcap_price_predict', 'engine': 'lightwood', 'project': 'rez_test_proj', 'version': 1, 'status': 'training', 'accuracy': None, 'predict': 'mcap_value', 'update_status': 'up_to_date', 'mindsdb_version': '23.5.4.1', 'error': None, 'select_data_query': 'SELECT mcap_date, asset_name, mcap_value FROM coin_mcap;', 'training_options': {'target': 'mcap_value'}, 'current_training_phase': 4, 'total_training_phases': 5, 'training_phase_name': 'Training model', 'tag': None, 'created_at': '2023-06-18 04:46:10.781935', 'training_time': 224}

In [39]:
# make predictions
mcap_row_limit = tbl.limit(3)

print('Making prediction on the first 3 rows of home_rentals table')
ret = mcap_model.predict(mcap_row_limit)

print(ret[['asset_name', 'mcap_date', 'mcap_value']])

Making prediction on the first 3 rows of home_rentals table
          asset_name            mcap_date   mcap_value
0           gamezone  2023-03-02 00:00:00  252573760.0
1  the_virtua_kolect  2023-03-02 00:00:00  164304656.0
2        alchemy_pay  2023-03-02 00:00:00  152188032.0


In [41]:
ret.describe()

,mcap_value,mcap_value_original,mcap_value_confidence,mcap_value_min,mcap_value_max
count,3.000000e+00,3.000000e+00,3.00,3.0,3.000000e+00
mean,1.896888e+08,7.288526e+07,0.98,0.0,2.562649e+09
std,5.479590e+07,9.348763e+07,0.00,0.0,5.479590e+07
min,1.521880e+08,1.586919e+07,0.98,0.0,2.525148e+09
25%,1.582463e+08,1.893927e+07,0.98,0.0,2.531206e+09
50%,1.643047e+08,2.200934e+07,0.98,0.0,2.537265e+09
75%,2.084392e+08,1.013933e+08,0.98,0.0,2.581399e+09
max,2.525738e+08,1.807773e+08,0.98,0.0,2.625534e+09


## get database

In [23]:
__def_db_name__ = "minds_test_db"

if __def_db_name__ not in [d.name for d in server.list_databases()]:
    _db = server.create_database(
        engine = "postgres",
        name = "minds_test_db",
        connection_args = {
            "host" : "172.17.0.1",
            "port" : "5432",
            "database":"minds_test_db",
            "user" : "nuwan",
            "password":"Kotchchi@7281"
        }
    )
_db = server.get_database('minds_test_db')
_db

RuntimeError: Can't connect to db: connection failed: FATAL:  database "minds_test_db" does not exist

In [177]:
mcap_model.get_status

<bound method Model.get_status of Model(mcap_price_predict, status=error)>

In [32]:
query = project.query('SELECT * FROM files.coin_mcap limit 3;')
print(query)


Query(SELECT * FROM files.coin_mcap limit 3;)


In [28]:
my_table = mysql_demo_db.get_table('warehouse.mcap_past')
my_table=my_table.filter(asset_symbol='nfy')
my_table=my_table.limit(3)
my_table.fetch()

,mcap_past_pk,uuid,data_source,asset_name,asset_symbol,alt_asset_id,currency,price_date,price_value,price_log_ror,...,volume_date,volume_size,volume_change,created_dt,created_by,created_proc,modified_dt,modified_by,modified_proc,deactivate_dt
0,1,6453adc74a6e0a5e3a77a792,coingecko,non_fungible_yearn,nfy,non-fungible-yearn,usd,2023-03-01 00:00:00.000000,0.760072,None,...,2023-03-01 00:00:00.000000,19706.965408,None,2023-05-05 07:18:09.962638,farmraider,wrangler_assets_etl_CryptoMarket function <nos...,None,None,None,None
1,2,6453adc74a6e0a5e3a77a7a2,coingecko,non_fungible_yearn,nfy,non-fungible-yearn,usd,2023-03-02 00:00:00.000000,0.775617,None,...,2023-03-02 00:00:00.000000,20273.635916,None,2023-05-05 07:18:09.962638,farmraider,wrangler_assets_etl_CryptoMarket function <nos...,None,None,None,None
2,3,6453adc74a6e0a5e3a77a7af,coingecko,non_fungible_yearn,nfy,non-fungible-yearn,usd,2023-03-03 00:00:00.000000,0.781697,None,...,2023-03-03 00:00:00.000000,21408.025024,None,2023-05-05 07:18:09.962638,farmraider,wrangler_assets_etl_CryptoMarket function <nos...,None,None,None,None
